In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
import torch

import wandb
from huggingface_hub import login, HfApi, create_repo
from pathlib import Path
# from datasets import Dataset, DatasetDict
# from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training


/home/012/r/rx/rxh210037/.conda/envs/llmenv/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/home/012/r/rx/rxh210037/.conda/envs/llmenv/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may

In [ ]:
#Wandb & Huggingface keys
hf_token = "***REMOVED***"
wandb_api_key = "***REMOVED***"

In [3]:
# Intitialize Weights & Biases
if wandb_api_key:
    wandb.login(key=wandb_api_key)
    print("Successfully logged in to WANDB!")
else:
    print("No wandb key provided. Skipping wandb login.")

if hf_token:
    
    # Log in to Hugging Face
    login(token=hf_token)
    print("Successfully logged in to Hugging Face!")
else:
    print("Hugging Face token not found in notebook secrets.")


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/012/r/rx/rxh210037/.netrc
wandb: Currently logged in as: reyhaneh-rhp7 (reyhaneh-rhp7-university-of-texas-at-dallas) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Successfully logged in to WANDB!
Successfully logged in to Hugging Face!


In [4]:
wandb.init(project="PartC_Training_Instruction_Model", name="PartC_Training_Instruction_Model")  


In [ ]:
data_folder = Path("***/Part C/jigsaw-agile-community-rules")
train_Path = data_folder / "train.csv"
test_Path = data_folder / "test.csv"
train = pd.read_csv(train_Path)
test = pd.read_csv(test_Path)
train.shape, test.shape

((2029, 9), (10, 8))

In [6]:
train["rule_violation"] = train["rule_violation"].map({0: "complies", 1: "violates"})
train.head()


,row_id,body,rule,subreddit,positive_example_1,positive_example_2,negative_example_1,negative_example_2,rule_violation
0,0,Banks don't want you to know this! Click here ...,"No Advertising: Spam, referral links, unsolici...",Futurology,If you could tell your younger self something ...,hunt for lady for jack off in neighbourhood ht...,Watch Golden Globe Awards 2017 Live Online in ...,"DOUBLE CEE x BANDS EPPS - ""BIRDS""\n\nDOWNLOAD/...",complies
1,1,SD Stream [ ENG Link 1] (http://www.sportsstre...,"No Advertising: Spam, referral links, unsolici...",soccerstreams,[I wanna kiss you all over! Stunning!](http://...,LOLGA.COM is One of the First Professional Onl...,#Rapper \n🚨Straight Outta Cross Keys SC 🚨YouTu...,[15 Amazing Hidden Features Of Google Search Y...,complies
2,2,Lol. Try appealing the ban and say you won't d...,No legal advice: Do not offer or request legal...,pcmasterrace,Don't break up with him or call the cops. If ...,It'll be dismissed: https://en.wikipedia.org/w...,Where is there a site that still works where y...,Because this statement of his is true. It isn'...,violates
3,3,she will come your home open her legs with an...,"No Advertising: Spam, referral links, unsolici...",sex,Selling Tyrande codes for 3€ to paypal. PM. \n...,tight pussy watch for your cock get her at thi...,NSFW(obviously) http://spankbang.com/iy3u/vide...,Good News ::Download WhatsApp 2.16.230 APK for...,violates
4,4,code free tyrande --->>> [Imgur](http://i.imgu...,"No Advertising: Spam, referral links, unsolici...",hearthstone,wow!! amazing reminds me of the old days.Well...,seek for lady for sex in around http://p77.pl/...,must be watch movie https://sites.google.com/s...,We're streaming Pokemon Veitnamese Crystal RIG...,violates


In [58]:
train_df, val_df = train_test_split(train, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(val_df, test_size=0.5, random_state=42)


In [59]:

dataset_prepared_path = "./prepared_data/"
output_dir = "./outputs_qwen_ruleviolation/"
wandb_project = "PartC_Training_Instruction_Model"
base_model = "Qwen/Qwen1.5-7B"
# hf_profile = "your_hf_username"


In [60]:
BASE_PROMPT = (
    "You are a rule compliance analyst. "
    "Your task is to determine whether the following Reddit comment complies with the subreddit rule or violates it. "
    "Only respond with 'complies' or 'violates'.\n\n"
)
COMPLETE_PHRASE = "Decide if this example complies or violates the rule."

# Build the full user prompt per example
def build_prompt(row):
    return f"""
{BASE_PROMPT}
Subreddit: r/{row['subreddit']}
Rule: {row['rule']}

Positive Examples:
1) {row['positive_example_1']}
{COMPLETE_PHRASE}

2) {row['positive_example_2']}
{COMPLETE_PHRASE}

Negative Examples:
1) {row['negative_example_1']}
{COMPLETE_PHRASE}

2) {row['negative_example_2']}
{COMPLETE_PHRASE}

---
Comment: {row['body']}
{COMPLETE_PHRASE}
""".strip()

In [61]:
def df_to_axolotl_format(df):
    return df.apply(
        lambda r: {
            "messages": [
                {"role": "user", "content": build_prompt(r)},
                {"role": "assistant", "content": r["rule_violation"]}
            ]
        },
        axis=1
    ).tolist()

# Build inference format (no assistant replies)
def df_to_inference_format(df):
    return df.apply(
        lambda r: {
            "messages": [
                {"role": "user", "content": build_prompt(r)}
            ]
        },
        axis=1
    ).tolist()

In [62]:
import json
output_dir = Path("prepared_data")
output_dir.mkdir(exist_ok=True)

# Convert train and validation sets
train_data = df_to_axolotl_format(train_df)
val_data = df_to_axolotl_format(val_df)
test_data= df_to_axolotl_format(test_df)

test_inference_data = df_to_inference_format(test)

# Save to JSONL
def save_jsonl(data, filename):
    path = output_dir / filename
    with open(path, "w", encoding="utf-8") as f:
        for item in data:
            f.write(json.dumps(item) + "\n")
    print(f" Saved {len(data)} records to {path}")

save_jsonl(train_data, "train.jsonl")
save_jsonl(val_data, "validation.jsonl")
save_jsonl(test_data, "test.jsonl")
save_jsonl(test_inference_data, "test_inference.jsonl")

 Saved 1623 records to prepared_data/train.jsonl
 Saved 203 records to prepared_data/validation.jsonl
 Saved 203 records to prepared_data/test.jsonl
 Saved 10 records to prepared_data/test_inference.jsonl


In [12]:
dataset_prepared_path = "./prepared_data/"
output_dir = "./outputs_qwen_ruleviolation/"
wandb_project = "PartC_Training_Instruction_Model"
base_model = "Qwen/Qwen1.5-7B"
# hf_profile = "your_hf_username"a/


In [81]:
yaml_template = f"""
seed: 42
torch_seed: 42

datasets:
  - path: {dataset_prepared_path}
    type: chat_template
    field_messages: messages
    chat_template: tokenizer_default
    train_on_split: train

test_datasets:
  - path: {dataset_prepared_path}
    type: chat_template
    field_messages: messages
    chat_template: tokenizer_default
    split: test
    
train_on_inputs: false

base_model: {base_model}
load_in_4bit: true
adapter: qlora
lora_r: 64
lora_alpha: 128
lora_dropout: 0.05
lora_target_linear: true

load_best_model_at_end: true
eval_strategy: epoch
save_strategy: epoch
metric_for_best_model: eval_loss
greater_is_better: false

micro_batch_size: 2
gradient_accumulation_steps: 8
gradient_checkpointing: true
fp16: true

learning_rate: 2e-4
lr_scheduler: cosine
num_epochs: 3
optimizer: paged_adamw_8bit
warmup_ratio: 0.1

output_dir: {output_dir}
report_to: wandb
wandb_project: {wandb_project}
"""

with open("config_partc.yaml", "w") as f:
    f.write(yaml_template)

print(" YAML config saved as config_partc.yaml")


 YAML config saved as config_partc.yaml


In [76]:
!axolotl preprocess config_partc.yaml --debug




     #@@ #@@      @@# @@#
    @@  @@          @@  @@           =@@#                               @@                 #@    =@@#.
    @@    #@@@@@@@@@    @@           #@#@=                              @@                 #@     .=@@
      #@@@@@@@@@@@@@@@@@            =@# @#     ##=     ##    =####=+    @@      =#####+  =#@@###.   @@
    @@@@@@@@@@/  +@@/  +@@          #@  =@=     #@=   @@   =@#+  +#@#   @@    =@#+  +#@#   #@.      @@
    @@@@@@@@@@  ##@@  ##@@         =@#   @#      =@# @#    @@      @@   @@    @@      #@   #@       @@
     @@@@@@@@@@@@@@@@@@@@          #@=+++#@=      =@@#     @@      @@   @@    @@      #@   #@       @@
                                  =@#=====@@     =@# @#    @@      @@   @@    @@      #@   #@       @@
    @@@@@@@@@@@@@@@@  @@@@        #@      #@=   #@=  +@@   #@#    =@#   @@.   =@#    =@#   #@.      @@
                                 =@#       @#  #@=     #@   =#@@@@#=    +#@@=  +#@@@@#=    .##@@+   @@
    @@@@  @@@@@@@@@@@@@@@@

[2025-10-22 13:54:

In [82]:
!accelerate launch -m axolotl.cli.evaluate config_partc.yaml


The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
[2025-10-22 14:13:15,028] [INFO] [axolotl.cli.config] config:
{
  "activation_offloading": false,
  "adapter": "qlora",
  "axolotl_config_path": "config_partc.yaml",
  "base_model": "Qwen/Qwen1.5-7B",
  "base_model_config": "Qwen/Qwen1.5-7B",
  "batch_size": 16,
  "bf16": false,
  "capabilities": {
    "bf16": true,
    "compute_capability": "sm_86",
    "fp8": false,
    "n_gpu": 1,
    "n_node": 1
  },
  "context_parallel_size": 1,
  "dataloader_num_workers": 1,
  "dataloader_pin_memory": true,
  "dataloader_prefetch_factor": 256,
  "dataset_num_proc": 24,
  "datasets": [
    {
      "chat_template": "tokeni

In [ ]:
#train_loss: 1.8255614042282104
#eval_loss: 1.7828916311264038

In [83]:
wandb.finish()

wandb: ERROR The nbformat package was not found. It is required to save notebook history.
